In [1]:
import pandas as pd
import os
import numpy as np



In [2]:
seg_folder = 'outputs/metrics_ondri_cerebra/'
metric_df = []
seg_files = os.listdir(seg_folder)
for file in seg_files:
    metric_df.append(pd.read_csv(seg_folder + file))

metric_df = pd.concat(metric_df)
metric_df['SUBJECT'] = [x.split('.')[:-1][0] for x in seg_files]


metric_df = metric_df.dropna(axis=1, how='all')
clinical_df = pd.read_csv('data/summary/ONDRI_summary.csv')
clinical_df = clinical_df.rename(columns={'COHORT': 'DIAGNOSIS'})
merged = pd.merge(clinical_df[['SUBJECT', 'DIAGNOSIS']], metric_df, on='SUBJECT')

In [4]:
ondri_demographic = []
for diagnosis in ['ADMCI','ALS', 'FTD', 'PD']:
    path = 'data/summary/ONDRI/Clinical_Release/%s/%s_01_Clinical_Aggregated.csv' % (diagnosis, diagnosis)
    tmp = pd.read_csv(path)
    ondri_demographic.append(tmp)
ondri_demographic= pd.concat(ondri_demographic)
ondri_demographic = ondri_demographic[['SUBJECT', 'AGE', 'SEX', 'MOCA_TOTAL']]
ondri_demographic = ondri_demographic.rename(columns={'AGE':'Age', 'SEX':'Gender', 'MOCA_TOTAL':'MOCA'})
ondri_demographic['Gender'] = ondri_demographic['Gender'].map({'Male':0, 'Female':1})


In [5]:
cohort = pd.merge(merged[['SUBJECT', 'DIAGNOSIS']], ondri_demographic, on='SUBJECT', how='left')


cohort['MOCA'][cohort['MOCA'] == 'M_PI'] = np.nan
cohort['MOCA'] = cohort['MOCA'].astype(float)
print('%s subjects have missing MOCA' % (cohort['MOCA'].isna().sum()))

means =cohort.drop(columns=['SUBJECT']).groupby('DIAGNOSIS').mean()
stds = cohort.drop(columns=['SUBJECT']).groupby('DIAGNOSIS').std()
print('%s are healthy controls' % str(np.sum(cohort['DIAGNOSIS']== 'HC')))

printout_age = ['%s (%s)' % (x, y) for x, y in zip(np.round(means['Age'], 1).astype(str), np.round(stds['Age'], 1).astype(str))]
printout_moca = ['%s (%s)' % (x, y) for x, y in zip(np.round(means['MOCA'], 1).astype(str), np.round(stds['MOCA'], 1).astype(str))]
printout_gender = ['%s' % x for x in np.round(means['Gender']*100, 1).astype(str)]
printout_N = [x for x in cohort[['DIAGNOSIS', 'Age']].groupby('DIAGNOSIS').count()['Age'].astype(str)]

printout = pd.DataFrame()
printout['DIAGNOSIS'] = means.index
printout['Age mean (SD)'] = printout_age
printout['Gender % female'] = printout_gender
printout['MOCA mean (SD)'] = printout_moca
printout['N'] = printout_N
printout['N'] = printout['N'].astype(int)

printout = printout.sort_values(by='N', ascending=False)
printout = printout[printout['DIAGNOSIS'] != 'Mixed']
printout.to_csv('outputs/demographic_summary.csv', index=False)


372 subjects have missing MOCA
18 are healthy controls


/tmp/ipykernel_22572/973364534.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cohort['MOCA'][cohort['MOCA'] == 'M_PI'] = np.nan
